In [2]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import ipywidgets as widgets
import serial
import time
from picoscope import ps5000a
import picosdk
from picosdk.discover import find_all_units
import serial.tools.list_ports as port_list
import chipwhisperer as cw

# Check if the Picoscope is connected
scopes = find_all_units()
for scope in scopes:
    print("Working with:")
    print(scope.info)
    scope.close()
ports = list(port_list.comports())
for p in ports:
    print (p)


Working with:
UnitInfo(driver=<picosdk.ps5000a.Ps5000alib object at 0x7fa85d457070>, variant=b'5244D', serial=b'KU687/0175')
/dev/ttyS4 - n/a


**PicoScope setup**

In [3]:
ps = ps5000a.PS5000a()
print("Found the following picoscope:")
print(ps.getAllUnitInfo())

# Since target runnning at 10 MHz and AES requires from trigger
obs_duration = 3.225E-6
# Sample at least 516 points within that window
sampling_interval = obs_duration / 1260
# Configure timebase
(actualSamplingInterval, nSamples, maxSamples) = ps.setSamplingInterval(sampling_interval, obs_duration)
print("Nsamples : ", nSamples)
print("Sampling interval = %f us" % (actualSamplingInterval*nSamples*1E6))

# 50mV range on channel A, AC coupled, 20 MHz BW limit
ps.setChannel('A', 'AC', 0.05, 0.0, enabled=True, BWLimited=True)
# Channel B is trigger
ps.setChannel('B', 'DC', 10.0, 0.0, enabled=True)
ps.setSimpleTrigger('B', 2.0, 'Rising', timeout_ms=2000, enabled=True)

Found the following picoscope:
DriverVersion                 : PS5000A Linux Driver, 2.1.113.3501
USBVersion                    : 3.0
HardwareVersion               : 1
VariantInfo                   : 5244D
BatchAndSerial                : KU687/0175
CalDate                       : 26Apr23
KernelVersion                 : 0.0
DigitalHardwareVersion        : 1
AnalogueHardwareVersion       : 1
PicoFirmwareVersion1          : 1.7.15.0
PicoFirmwareVersion2          : 1.2.34.0
Nsamples :  1612
Sampling interval = 3.224000 us


**CW305 setup**

In [4]:
TARGET_PLATFORM = 'CW305_100t'
fpga_id = '100t'
target = cw.target(None, cw.targets.CW305, fpga_id=fpga_id, force = True)

print(target.is_programmed())

target.vccint_set(1.0)
target.pll.pll_enable_set(True)         # enable PLL chip
target.pll.pll_outenable_set(False, 0)  # disable PLL 0
target.pll.pll_outenable_set(True, 1)   # enable PLL 1
target.pll.pll_outenable_set(False, 2)  # disable PLL 2
target.pll.pll_outfreq_set(10E6, 1)     # PLL1 frequency set to 10 MHz
# 1 ms is plenty idling time --> maybe not useful with picoscope (?) 
target.clksleeptime = 1 

True


**Capture loop**

In [5]:
def pico_capture():
    # Arm the picoscope
    ps.runBlock()
    time.sleep(0.05)
    # Trigger the encryption on Target
    target.fpga_write(target.REG_USER_LED, [0x01])
    target.usb_trigger_toggle()
    ps.waitReady()
    # Capture the trace 
    data = ps.getDataV('A', nSamples, returnOverflow=False)
    return data

In [6]:
from tqdm.notebook import tnrange
project_file = "sca_test_CW305.cwp"
project = cw.create_project(project_file, overwrite=True)

from Crypto.Cipher import AES
from chipwhisperer.common.traces import Trace

ktp = cw.ktp.Basic()
# Initialize cipher to verify DUT result:
key, text = ktp.next()
cipher = AES.new(bytes(key), AES.MODE_ECB)
print("Key: ", key)
#print("Plaintext: ", text)

N = 1000        # Number of traces
traces = []
textin = []
keys = []
data_mV = []

target.fpga_write(target.REG_CRYPT_KEY, key)

# Dummy capture call due to bug of using AC coupling
pico_capture()

for i in tnrange(N, desc='Capturing traces'):
    
    # Write plaintext to target
    target.fpga_write(target.REG_CRYPT_TEXTIN, text)

    # Capture the trace 
    data = pico_capture()
    traces.append(np.array(data))
    data_mV.append(np.array(data)*1E3)

    # Organize and store data
    response = target.fpga_read(target.REG_CRYPT_CIPHEROUT, 16)
    trace_i = Trace(np.array(data), text, response[::-1], key)
    project.traces.append(trace_i)
    
    # Sanity check with expected ciphertext
    #print("Response fpga: ",[ hex(el) for el in response[::-1]])
    #print("Expected instead: ", [hex(el) for el in cipher.encrypt(bytes(text))])
    #assert (list(response[::-1]) == list(cipher.encrypt(bytes(text)))), "Incorrect encryption result!\nGot {}\nExp {}\n".format(list(response), list(text))
    
    key, text = ktp.next() 
    textin.append(text)
    keys.append(key)

project.save()
target.dis()

Key:  CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')


Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

** Plotting trace **

In [7]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()
p = figure(plot_width=800)

xrange = range(len(traces[0]))
p.line(xrange, traces[4], line_color="red")
show(p)

Loading BokehJS ...

** Performing AES CPA attack **

In [8]:
import chipwhisperer.analyzer as cwa
attack = cwa.cpa(project, cwa.leakage_models.last_round_state_diff)
cb = cwa.get_jupyter_callback(attack)
results = attack.run(cwa.get_jupyter_callback(attack, 25))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
PGE=,19,14,245,68,181,108,7,143,222,61,55,78,237,31,1,201
0,AB0.154,C10.213,D20.214,280.173,320.129,E60.191,310.160,C90.162,990.145,630.182,1F0.167,6D0.154,B70.199,BE0.181,5E0.171,940.200
1,CB0.135,100.144,280.150,F30.136,300.128,A40.128,DF0.128,C70.135,C90.140,2E0.152,5C0.151,5F0.131,7F0.132,800.166,0C0.135,900.163
2,940.133,FC0.125,550.135,0A0.127,1C0.125,520.128,FA0.126,DF0.132,2A0.136,5A0.139,C90.139,730.125,4E0.129,E60.132,E70.131,600.146
3,7D0.131,900.123,230.133,4F0.123,420.124,CE0.127,800.125,180.127,6C0.135,1D0.131,700.137,C70.123,5E0.121,050.127,630.129,B50.127
4,300.126,0C0.123,250.129,730.123,770.120,2C0.126,990.124,670.123,C60.134,430.127,E90.134,130.121,FE0.121,750.126,000.121,0F0.126
5,5C0.126,A40.122,AA0.123,6A0.121,540.120,3D0.125,220.124,C00.123,A90.126,870.121,CE0.130,B80.121,B10.121,2C0.123,FD0.121,080.121
6,4E0.122,EC0.119,450.121,C90.121,EA0.120,110.123,B90.120,440.122,900.124,7B0.120,B90.125,BC0.121,4F0.120,280.115,390.119,E10.120
7,DB0.121,EE0.118,CB0.118,B40.120,E10.120,470.119,250.120,3C0.118,420.124,080.118,1A0.125,740.120,CB0.120,710.112,500.118,2A0.116
8,640.120,050.117,910.117,B00.117,450.119,AF0.115,960.117,3D0.114,660.118,DD0.117,FE0.124,2B0.119,9E0.119,010.112,4C0.117,5F0.115


In [9]:

from chipwhisperer.analyzer.attacks.models.aes.key_schedule import key_schedule_rounds
recv_lastroundkey = [kguess[0][0] for kguess in results.find_maximums()]
recv_key = key_schedule_rounds(recv_lastroundkey, 10, 0)
for subkey in recv_key:
    print(hex(subkey))
key=list(key)
assert (key == recv_key), "Failed to recover encryption key!\nGot {}\nExp {}\n".format(recv_key, key)

0x21
0xf9
0xed
0x37
0x67
0xf6
0xa6
0x37
0x98
0x7d
0xe6
0xc1
0xcb
0x2b
0x40
0x84


AssertionError: Failed to recover encryption key!
Got [33, 249, 237, 55, 103, 246, 166, 55, 152, 125, 230, 193, 203, 43, 64, 132]
Exp [43, 126, 21, 22, 40, 174, 210, 166, 171, 247, 21, 136, 9, 207, 79, 60]


In [10]:
leak_model = cwa.leakage_models.sbox_output
attack = cwa.cpa(project, leak_model)
results = attack.run(cwa.get_jupyter_callback(attack, 25))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
PGE=,228,251,33,146,163,203,156,248,44,113,19,35,219,191,123,250
0,D40.158,860.147,A40.146,4F0.141,210.143,050.133,6B0.137,B00.142,940.148,180.140,BD0.131,CE0.139,D60.144,8B0.146,F40.146,DA0.130
1,CC0.134,4F0.132,D30.133,FC0.128,9D0.136,F60.129,D10.135,1F0.130,AA0.146,820.132,F00.128,170.132,E60.137,B90.143,400.129,AA0.126
2,9B0.132,A60.128,800.124,F90.127,CB0.134,A20.127,CC0.130,A80.130,360.130,6B0.129,9D0.123,AA0.125,A60.131,9A0.137,F50.121,220.124
3,D80.127,D30.127,950.122,FB0.127,A60.131,980.123,840.128,B50.128,BF0.119,510.126,240.123,460.124,AD0.124,D50.131,BB0.119,950.121
4,890.124,030.124,B20.121,950.124,FA0.131,9C0.121,B60.128,F70.128,F20.117,7E0.121,0D0.121,FD0.124,B30.120,7D0.131,830.118,990.116
5,4C0.115,CB0.121,5F0.117,E10.123,D40.130,540.120,0D0.127,2F0.124,820.117,D50.120,8B0.121,190.121,D00.118,270.131,490.117,5A0.116
6,380.115,8A0.117,470.116,A50.123,3E0.123,BE0.118,A10.125,E40.122,510.116,6E0.119,560.120,D90.121,F50.118,E10.129,A30.115,AC0.115
7,970.114,F90.115,110.115,130.121,CC0.121,DE0.114,BA0.123,6C0.122,300.113,8C0.119,F80.119,900.120,B60.116,120.126,1E0.114,0C0.112
8,520.112,2D0.113,260.114,2B0.119,270.120,940.114,D90.121,580.121,7B0.112,B30.119,0F0.117,AD0.118,E80.112,630.126,880.111,B50.112
